In [1]:
import os

In [2]:
os.chdir(r'D:\subFinder')

In [3]:
## In this script we will train supervised models
# library imports
import pandas as pd
import gensim
from Codes.Supervised_Trainer import run_end_to_end
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.multiclass import OneVsRestClassifier
import plotly.express as px

In [4]:

## read the data
# sup_data_path = r"Data/Supervised_Sequences/dbCAN-PUL_07-01-2022.xlsx"
# new_path = r"D:\subFinder\Data\Supervised_Sequences\Table S1.xls"
# data = pd.read_excel(new_path)
# data["updated_substrate (09/01/2022)"] = data["updated_substrate (09/01/2022)"].str.strip()
# data["updated_substrate (09/01/2022)"] = data["updated_substrate (09/01/2022)"].str.strip()


# ## removing the catch all classes
# ## for example multiple substrates and others 
# old_data = pd.read_csv('Data/Supervised_Sequences/pul_seq_low_high_substr_year_corrected.tsv', sep = "\t")

# old_data["high_level_substr"] = old_data["high_level_substr"].str.strip()

# bad_puls = old_data[old_data["high_level_substr"].isin(["multiple_substrates", "mono/di/trisaccharide", "-", "human milk oligosaccharide", 
#                                             "glycoprotein", "plant polysaccharide", "cellobiose"])]["PULid"].values


# data = data[~data["PUL ID"].isin(bad_puls)]

# data = data[["PUL ID", "updated_substrate (09/01/2022)"]]
# data = pd.merge(data, old_data[["PULid","sig_gene_seq"]], left_on = ["PUL ID"], right_on = ["PULid"], how = "left")
# data.columns = ["PUL ID", "high_level_substr", "PULid", "sig_gene_seq"]

# data = data[~data["PUL ID"].isin(bad_puls)]


In [5]:
## read the data
# sup_data_path = r"Data/Supervised_Sequences/dbCAN-PUL_07-01-2022.xlsx"
# new_sup_data_with_nulls = pd.read_csv(r"D:\subFinder\Data\Supervised_Sequences\supervised_seq_with_null.tsv", sep = "\t", 
#                                       header = None)
# new_sup_data_with_nulls.columns = ["PUL ID", "sequence"]

In [6]:
# data = pd.merge(data, new_sup_data_with_nulls, on = ["PUL ID"], how = "left")
# data = data.drop(["sig_gene_seq"],1)
# data.columns = ['PUL ID', 'high_level_substr', 'PULid', 'sig_gene_seq']

In [7]:
# data.to_csv(r"Data/Supervised_Sequences/updated_data_filtered.csv", index = False)

In [8]:
data = pd.read_csv(r"D:\subFinder\Data\Supervised_Sequences\updated_data_filtered.csv")

In [9]:
data["high_level_substr"].value_counts()

capsule polysaccharide synthesis      106
pectin                                 39
xylan                                  35
beta-glucan                            21
alginate                               21
host glycan                            19
cellulose                              16
starch                                 16
chitin                                 15
galactomannan                          15
glycosaminoglycan                      14
arabinan                               12
galactan                               10
alpha-mannan                            8
beta-mannan                             8
carrageenan                             8
xyloglucan                              7
fructan                                 7
arabinoxylan                            6
agarose                                 6
arabinogalactan                         5
capsule polysaccharide degradation      4
beta-galactooligosaccharide             4
glucomannan                       

In [10]:
data = data[data["high_level_substr"] != "capsule polysaccharide synthesis"]

In [11]:
data["high_level_substr"].value_counts()

pectin                                39
xylan                                 35
beta-glucan                           21
alginate                              21
host glycan                           19
cellulose                             16
starch                                16
chitin                                15
galactomannan                         15
glycosaminoglycan                     14
arabinan                              12
galactan                              10
alpha-mannan                           8
beta-mannan                            8
carrageenan                            8
xyloglucan                             7
fructan                                7
arabinoxylan                           6
agarose                                6
arabinogalactan                        5
capsule polysaccharide degradation     4
beta-galactooligosaccharide            4
glucomannan                            3
glycogen                               3
alpha-glucan    

In [12]:
data.tail()

,PUL ID,high_level_substr,PULid,sig_gene_seq
406,PUL0065,xyloglucan,PUL0065,"1.B.14,8.A.46,null,null,GH43_12,null,GH5_4,HTH..."
407,PUL0066,xyloglucan,PUL0066,"null,1.B.14,null,null,GH43_12,GH5_4,null,HTH_A..."
408,PUL0067,xyloglucan,PUL0067,"GH2,GH5_4,null,8.A.46,1.B.14,GH31,HTH_AraC,GH9..."
409,PUL0215,xyloglucan,PUL0215,"GH31,GH35,1.B.14,GH95"
410,PUL0527,xyloglucan,PUL0527,"null,HATPase_c,null,9.B.174,GH3,GH2,GH31,HTH_A..."


In [13]:

# top_k = 10
K = 10
known_unknown = False


## Doc2Vec_DM model
# model_dm =  gensim.models.doc2vec.Doc2Vec.load(r"Embedding_Models_10_12//doc2vec_dm") 
# model_dbow = gensim.models.doc2vec.Doc2Vec.load(r"Embedding_Models_10_12//doc2vec_dbow") 
# model_cbow =gensim.models.word2vec.Word2Vec.load(r"Embedding_Models_10_12//word2vec_cbow") 
# model_sg =gensim.models.word2vec.Word2Vec.load(r"Embedding_Models_10_12//word2vec_sg") 
# model_fasttext_sg =gensim.models.word2vec.Word2Vec.load(r"Embedding_Models_10_12//fasttext_sg") 
# model_fasttext_cbow =gensim.models.word2vec.Word2Vec.load(r"Embedding_Models_10_12//fasttext_cbow") 


In [14]:
model_sg_50 = gensim.models.word2vec.Word2Vec.load(r"Embedding_Models_10_12//word2vec_sg_vec_50") 

In [15]:
og = ["lstm_with_attention"]

In [16]:
overall_catch = []
for top_k in tqdm(range(4,5)):
    for featurizer in og:
        print("Currently running for featurizer "+ featurizer + " with " + str(top_k) + " number of classes.")
        if featurizer == "countvectorizer":
            avg_acc, avg_class_acc, std_err_avg_acc, std_err_avg_classwise_acc, overall_report,  model, params_best, fig, fig2, fig3 = run_end_to_end(top_k, data, featurizer, K, known_unknown, None)
        elif featurizer == "doc2vec_dbow":
            avg_acc, avg_class_acc, std_err_avg_acc, std_err_avg_classwise_acc, overall_report,  model, params_best, fig, fig2, fig3 = run_end_to_end(top_k, data, featurizer, K, known_unknown, model_dbow)
        elif featurizer == "doc2vec_dm":
            avg_acc, avg_class_acc, std_err_avg_acc, std_err_avg_classwise_acc, overall_report,  model, params_best, fig, fig2, fig3 = run_end_to_end(top_k, data, featurizer, K, known_unknown, model_dm)
        elif featurizer == "word2vec_cbow":
            avg_acc, avg_class_acc, std_err_avg_acc, std_err_avg_classwise_acc, overall_report,  model, params_best, fig, fig2, fig3 = run_end_to_end(top_k, data, featurizer, K, known_unknown, model_cbow)            
        elif featurizer == "word2vec_sg":
            avg_acc, avg_class_acc, std_err_avg_acc, std_err_avg_classwise_acc, overall_report,  model, params_best, fig, fig2, fig3 = run_end_to_end(top_k, data, featurizer, K, known_unknown, model_sg)
        elif featurizer == "fasttext_sg":
            avg_acc, avg_class_acc, std_err_avg_acc, std_err_avg_classwise_acc, overall_report,  model, params_best, fig, fig2, fig3 = run_end_to_end(top_k, data, featurizer, K, known_unknown, model_fasttext_sg)      
        elif featurizer == "fasttext_cbow":
            avg_acc, avg_class_acc, std_err_avg_acc, std_err_avg_classwise_acc, overall_report,  model, params_best, fig, fig2, fig3 = run_end_to_end(top_k, data, featurizer, K, known_unknown, model_fasttext_cbow)  
            
        elif featurizer == "lstm_with_attention": 
            avg_acc, avg_class_acc, std_err_avg_acc, std_err_avg_classwise_acc, overall_report,  model, params_best, fig, fig2, fig3 = run_end_to_end(top_k, data, featurizer, K, known_unknown, model_sg_50)
        
        elif featurizer == "just_attention": 
            avg_acc, avg_class_acc, std_err_avg_acc, std_err_avg_classwise_acc, overall_report,  model, params_best, fig, fig2, fig3 = run_end_to_end(top_k, data, featurizer, K, known_unknown, model_sg_50)        
        
        elif featurizer == "vanilla_lstm": 
            avg_acc, avg_class_acc, std_err_avg_acc, std_err_avg_classwise_acc, overall_report,  model, params_best, fig, fig2, fig3 = run_end_to_end(top_k, data, featurizer, K, known_unknown, model_sg_50)        
        
        else:
            pass
        
        overall_catch.append([top_k, featurizer, avg_acc, avg_class_acc, std_err_avg_acc, std_err_avg_classwise_acc, 
                             overall_report["precision"], overall_report["recall"], overall_report["f1-score"]])
#         print(avg_acc, avg_class_acc)
        plt.close('all')

  0%|          | 0/1 [00:00<?, ?it/s]

Currently running for featurizer lstm_with_attention with 4 number of classes.



0it [00:00, ?it/s]
1it [00:59, 59.04s/it]
2it [01:39, 47.96s/it]
3it [03:04, 64.76s/it]
4it [04:09, 64.97s/it]
5it [04:58, 59.35s/it]
6it [05:37, 52.35s/it]
7it [06:19, 48.96s/it]
8it [07:02, 47.18s/it]
9it [07:43, 45.01s/it]
10it [08:26, 50.70s/it]


Currently running for featurizer just_attention with 4 number of classes.



0it [00:30, ?it/s]

KeyboardInterrupt



In [17]:
overall_catch = pd.DataFrame(overall_catch)

In [18]:
overall_catch.columns = ["num_substrates", "feature_method", "avg_accuracy", "avg_classwise_acc",
                         "std_err_avg_acc", "std_err_avg_classwise_acc", "avg_precision", "avg_recall", "avg_f1_score"]

In [19]:
overall_catch.head(10)

,num_substrates,feature_method,avg_accuracy,avg_classwise_acc,std_err_avg_acc,std_err_avg_classwise_acc,avg_precision,avg_recall,avg_f1_score
0,4,lstm_with_attention,0.914394,0.920833,0.084974,0.085797,0.942917,0.920833,0.922996


In [20]:
enkf_k_fold_metrics = pd.read_csv(r"D:\Proposal_Defense\kf_probit\enkf_subFinder_validation_accuracies_over_folds.csv")

In [21]:
enkf_k_fold_metrics.head()

,fold_update_index,variable,value
0,1,acc_enkf_15_20,1.000000
1,2,acc_enkf_15_20,0.916667
2,3,acc_enkf_15_20,0.833333
3,4,acc_enkf_15_20,0.916667
4,5,acc_enkf_15_20,0.750000


In [22]:
enkf_k_fold_metrics.columns = ["fold_update_index", "model", "Accuracy"]

In [23]:
enkf_k_fold_metrics.groupby(["model"])["Accuracy"].mean()

model
acc_ann_15        0.915909
acc_ann_20        0.915152
acc_enkf_15_20    0.879545
Name: Accuracy, dtype: float64

In [24]:
enkf_k_fold_metrics.groupby(["model"])["Accuracy"].std()

model
acc_ann_15        0.087874
acc_ann_20        0.080476
acc_enkf_15_20    0.092952
Name: Accuracy, dtype: float64